# Ray Tune - Basic Tune Concepts and Steps: Warming Up with Ray Tune Hyperparameter Tuning

© 2019-2021, Anyscale. All Rights Reserved

![Anyscale Academy](../images/AnyscaleAcademyLogo.png)

This lesson introduces Ray tune's key concepts using a trivial examples. This example is derived from [Ray Tune basic example](https://docs.ray.io/en/latest/tune/examples/tune_basic_example.html). Basically, there are three basic steps or Ray Tune pattern for you as a newcomer to get started with using Ray Tune.

 1. Setup your config space and define your trainable and objective function
 2. Use tune to execute your training, supplying the appropriate arguments including: search space, [search algorithms](https://docs.ray.io/en/latest/tune/api_docs/suggestion.html#blendsearch) or [trial schedulers](https://docs.ray.io/en/latest/tune/api_docs/schedulers.html#tune-schedulers)
 3. Examine analyse the results
 
 ![](https://docs.ray.io/en/latest/_images/tune-workflow.png)


See also the [Hyperparameter Tuning References](References-Hyperparameter-Tuning.ipynb) notebook and the [Tune documentation](http://tune.io), in particular, the [API reference](https://docs.ray.io/en/latest/tune/api_docs/overview.html). 


### Install Ray Tune
[Ray Tune](https://docs.ray.io/en/master/installation.html#official-releases) requires a separate install. 

In [ ]:
!pip install 'ray[tune]'

### 1. Setup training using Trainable APIs

In [ ]:
import time

import ray
from ray import tune

Let's define our objective function

In [ ]:
def evaluation_fn(step, width, height):
    time.sleep(0.1)
    return (0.1 + width * step / 100)**(-1) + height * 0.1

Next, we define a Trainable used by Tune using Tune's [Functional API](https://docs.ray.io/en/latest/tune/api_docs/trainable.html#function-api)

In [ ]:
def easy_objective(config):
    # fetch our Hyperparameters sent as arguments
    width, height = config["width"], config["height"]
    # Iterate over number of steps
    for step in range(config["steps"]):
        # Iterative training function - can be any arbitrary training procedure
        # Here our objective function is the evaluation_fn
        intermediate_score = evaluation_fn(step, width, height)
        # Feed the score back back to Tune.
        tune.report(iterations=step, mean_loss=intermediate_score)

In [ ]:
ray.init(ignore_reinit_error=True)

### Step 2. Use tune API to execute tuning

This will do a grid search over the `activation` parameter. This means
that each of the two values (`relu` and `tanh`) will be sampled once
for each sample (`num_samples`). We end up with 2 * N = 2N samples, where is `num_samples`
The `width` and `height` parameters are sampled randomly.
`steps` is a constant parameter.

The `tune.run(..)` API returns a large [analysis](https://docs.ray.io/en/latest/tune/api_docs/analysis.html#analysis-tune-analysis) object. 

In [ ]:
analysis = tune.run(
    easy_objective,
    metric="mean_loss",
    mode="min",
    num_samples=5,
    # Define our hypyerparameter search space
    config={
        "steps": 5,
        "width": tune.uniform(0, 20),
        "height": tune.uniform(-100, 100),
        "activation": tune.grid_search(["relu", "tanh"]),
    },
    verbose=1
)

### Step 3. Analyse the results

In [ ]:
print("Best hyperparameters found were: ", analysis.best_config)

Alternatively, you can examine a Pandas dataframe

In [ ]:
analysis.results_df.head(5)

In [ ]:
analysis.trials

In [ ]:
analysis.dataframe(metric="mean_loss", mode="min").head(5)

In [ ]:
ray.shutdown()